In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

import xgboost as xgb
from sklearn.model_selection import train_test_split

param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 10,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
print(predict)
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('submit_xgb.csv')


D:\anocconda3-5.20\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-auc:0.863206	valid-auc:0.697916
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[25]	train-auc:0.969418	valid-auc:0.770435
[50]	train-auc:0.973208	valid-auc:0.779876
[75]	train-auc:0.983537	valid-auc:0.781294
[100]	train-auc:0.987372	valid-auc:0.780749
[125]	train-auc:0.990794	valid-auc:0.783695
[150]	train-auc:0.993722	valid-auc:0.788279
[175]	train-auc:0.996632	valid-auc:0.79199
[200]	train-auc:0.99797	valid-auc:0.797446
[225]	train-auc:0.998886	valid-auc:0.801702
[250]	train-auc:0.999317	valid-auc:0.807923
[275]	train-auc:0.999641	valid-auc:0.811088
[300]	train-auc:0.999838	valid-auc:0.813598
[325]	train-auc:0.999883	valid-auc:0.814362
[350]	train-auc:0.999937	valid-auc:0.815453
[375]	train-auc:0.999955	valid-auc:0.817745
[400]	train-auc:0.999991	valid-auc:0.818837
[425]	train-auc:1	valid-auc:0.819819
[450]	train-auc:1	valid-auc:0.821019
[475]	train-auc:1	valid-auc:0.821456
[500